Importing libraries to use

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#imgArr = cv2.imread("dataset/0/Training_31410334.jpg")
#imgArr.shape # check size and layers in image

In [ ]:
#plt.imshow(imgArr) #show image

In [ ]:
data_folder="drive//MyDrive//Colab Notebooks//SSN//dataset" # dataset folder

In [ ]:
folders=['0','1','2','3','4','5','6'] #folders in dataset folder
folders=['0','3','5'] #folders in dataset folder

# Read all images and convert them into array

In [ ]:
imgSize=224 #we will change size because mobileNet accepts 224*224 in rgb
training_data=[] # a data list
def create_training_data():

    for folder in folders:
        path= os.path.join(data_folder,folder)
        print(os.path.join(data_folder,folder))
        #id=int(folder)
        id= folders.index(folder) # get id for each image, id is index of each emotion folder
        for img in os.listdir(path):
            try:
                print(os.path.join(path,img))
                imgArray= cv2.imread(os.path.join(path,img))
                resizeArray= cv2.resize(imgArray,(imgSize,imgSize)) # resizing the images from 48*48 to 224*224
                training_data.append([resizeArray,id]) # push image and its id
            except Exception as e:
                pass


In [ ]:
create_training_data()

In [ ]:
print(len(training_data))

import pickle

with open("drive//MyDrive//Colab Notebooks//SSN//training_dataLess.txt", "wb") as fp:   #Pickling
  pickle.dump(training_data, fp)

6949


In [ ]:
print(len(training_data))
import pickle

with open("drive//MyDrive//Colab Notebooks//SSN//training_data.txt", "wb") as fp:   #Pickling
  pickle.dump(training_data, fp)

10775


In [ ]:
import pickle
with open("drive//MyDrive//Colab Notebooks//SSN//training_data.txt", "rb") as fp:   # Unpickling
  training_data = pickle.load(fp)
print(len(training_data))


10775


In [ ]:
import random
random.shuffle(training_data)
#shuffeling the data so that modle will not learn the sequence

#now split id and imageArray and save it in different list

In [ ]:
x=[]
y=[]

for features,lable in training_data:
    x.append(features)
    y.append(lable)

##learn 
X= np.array(x).reshape(-1,imgSize,imgSize,3) # converting it to 4 dimension

In [ ]:
X.shape

(5269, 224, 224, 3)

In [ ]:
#normalize the data
X=X/255.0;

In [ ]:
Y=np.array(y) #covert list into np array
Y.shape

(5269,)



```
# This is formatted as code
```

#train deep learning model by transfer learning

In [ ]:
#importing some other files
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model= tf.keras.applications.MobileNetV2() # Pre trained model

In [ ]:
model.summary()

# transfer learning - tuning, weights will start from last

In [ ]:
base_input = model.layers[0].input  #1st layer
base_output = model. layers[-2].output #last layer so that we can convert 1000 classess to 7 classess since our emotions are only 7

In [ ]:
base_output

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [ ]:
final_output = layers.Dense(128)(base_output) # adding new layer, after the output of global pooling layer
final_output = layers.Activation('relu')(final_output) #activation function
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7,activation='softmax')(final_output) # we have 7 classes/emotions #classification layer

In [ ]:
final_output = layers.Dense(128)(base_output) # adding new layer, after the output of global pooling layer
final_output = layers.Activation('relu')(final_output) #activation function
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(3,activation='softmax')(final_output) # we have 7 classes/emotions #classification layer

In [ ]:
final_output

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_5')>

In [ ]:
new_model= keras.Model(inputs= base_input,outputs= final_output)

In [ ]:
new_model.summary()

In [ ]:
new_model.compile(loss="sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [ ]:
new_model.fit(X,Y,epochs=20)

Epoch 1/25


In [ ]:
newmodel.save('drive//MyDrive//Colab Notebooks//SSN//trained_model.h5')